複習一些過往的code並慢慢整理自己的codebase，並且養成每日寫data science code的習慣!

## 目標: 將過往code文件化、模組化，使其重複使用。

## Day23~25- NLP: 基礎 text summarization 
1. Counter
2. tf-idf

In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
import warnings
import jieba

warnings.filterwarnings("ignore")
%matplotlib inline

## 步驟
1. 取得中文文章當作 raw data。
2. 基本中文文字處理。
    1. 將每一個wiki文章轉化成句子。
    2. 所有句子組合成一個corpus。
3. 將中文句子、文字向量化。
    1. 透過corpus去計算 Count Vector，也就是每一個word，出現頻率。

4. 根據向量化分數取得摘要。
    1. 輸入為數個句子(一篇短文)，挑選出 top n的句子作為摘要。
    2. 其中每一個句子會有重要性分數，根據其每一個word的分數加總而成。(此處需要注意，這要特別小心處理贅詞，但這先忽略跳過)

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
# 1. 使用關鍵字+中文wiki去取得文章當作 raw data。
url = 'https://zh.wikipedia.org/zh-tw/'

# 3種不同領域的各5個關鍵字
keywords = [
    # 資料科學領域
    '資料科學',
    '機器學習',
    '深度學習',
    '自然語言處理',
    '電腦視覺',
    # 軟體工程
    '軟體工程',
    '網頁開發者',
    '資訊工程',
    '資料庫',
    '網路',
    # 文本刊物
    '假新聞',
    '新聞',
    '期刊',
    '部落格',
    'YouTube'
]

因為一的wiki裡面有多個描述，可以將其分成多個句子，最後全部組成成一個corpus。

In [61]:
from collections import Counter

corpus = []
word2freq = Counter()
max_freq = 0

for keyword in keywords:
    # 1.1 從網址取得內容
    resp = requests.get(url + keyword)
    content = resp.content
    soup = BeautifulSoup(content)
    #print(soup)
    # 2. 從文字內容斷詞，這邊先不做太多處理，將其拆分成一句一句，不乾淨等還可以使用
    document_text = soup.find(id='bodyContent').text
    raw_document_text = []
    for sentence in document_text.split('\n'):
        if sentence:
            raw_document_text.append(sentence)
    #print(raw_document_text)
    # print(list(jieba.cut(raw_document_text)))
    # 3.
    for sentence in raw_document_text:
        words = list(jieba.cut(sentence))
        for word in words:
            word2freq[word] += 1
            max_freq = max(max_freq, word2freq[word])
        corpus.append(sentence)

In [62]:
# 詞頻: 作為權重計算用
for word in word2freq:
    word2freq[word] /= max_freq
    
word2freq

Counter({'維基': 0.008829747543837831,
         '百科': 0.00932719810968785,
         '，': 0.240268623305559,
         '自由': 0.0047257803755751775,
         '的': 0.2752145255565228,
         '全書': 0.00186543962193757,
         '（': 0.16440741201343118,
         '重新': 0.0019898022634000745,
         '導向': 0.0019898022634000745,
         '自資料': 0.0002487252829250093,
         '科學': 0.01305807735356299,
         '）': 0.16440741201343118,
         '跳至': 0.00373087924387514,
         '導覽': 0.00186543962193757,
         '搜尋': 0.00559631886581271,
         '此條目': 0.0008705384902375327,
         '需要': 0.004601417734112673,
         '擴充': 0.001119263773162542,
         '。': 0.14799154334038056,
         ' ': 1.0,
         '(': 0.026737967914438502,
         '2013': 0.008083571695062804,
         '年': 0.045268001492351696,
         '7': 0.006964307921900261,
         '月': 0.03270737470463873,
         '15': 0.007337395846287775,
         '日': 0.019400572068150726,
         ')': 0.026737967914438502,

In [106]:
# 4. 根據向量化分數取得摘要。
document_text = """資料科學（英語：data science）又稱數據科學，是一門利用資料（數據）學習知識的學科，其目標是通過從資料中提取出有價值的部分來生產資料產品[1]，學科範圍涵蓋了：資料取得、資料處理、資料分析等過程，舉凡與資料有關的科學均屬資料科學。

資料科學結合了諸多領域中的理論和技術，包括應用數學、統計、圖型識別、機器學習、資料視覺化、資料倉儲以及高效能計算。資料科學通過運用各種相關的資料來幫助非專業人士理解問題。 資料科學技術可以幫助我們如何正確的處理資料並協助我們在生物學、社會科學、人類學等領域進行研究調研。此外，資料科學也對商業競爭有極大的幫助[2]。

美國國家標準技術研究所於2015年發表七卷巨量資料參考框架（NIST Big Data Reference Architecture，NBDRA），於第一卷定義篇中將資料科學定為在理論科學、實驗科學和計算科學之後的第四科學範式[3][4]。1960年代，彼得·諾爾就首次提議要用「數據科學（data science、Datalogy）」來替代「計算機科學（computer science）」，後來在上世紀九十年代中期為國際分類社團聯盟所用。

1997年11月，吳建福 發表了題為「統計=資料科學？」的就任演講，[5] 因為他被任命為密西根大學的 H. C.卡弗教授。[6] 在這次演講中，他將統計工作描述為資料收集、資料建模和分析以及決策制定的三部曲。 在他的結論中，他開創了「資料科學」（而非「電腦科學」）這個術語的現代用法，並提倡將統計學重新命名為資料科學，而統計學家則應重新命名為資料科學家。[5] 後來，他發表了題為「統計=資料科學？」的演講，作為他1998年的首個 P. C. 馬哈拉諾比斯紀念演講。[7] 這些講座是為了紀念印度科學家、統計學家和印度統計學院​（英語）創始人P. C. 馬哈拉諾比斯​（英語）。

2001年，William S. Cleveland 提議將其設立為一個新的學科，吸收「計算在數據方面取得的進展」作為統計學的延伸。《Data Science Journal》及《The Journal of Data Science》分別於2002年與2003年發行；2005年9月，美國國家科學基金會發表了《長存的數位資料收集：使21世紀的研究與教育成為可能》，文中將數據科學家定義為「資訊與電腦科學家，資料庫與軟體工程師與程式設計師，跨學科專家，保管員以及專業注釋者，圖書館員，檔案館員和其他人員，這些人對數位資料收集的成功管理至關重要。」[8] 2008年 Jeff Hamerbatcher 與 DJ Patil circa 分別在 Facebook、LinkedIn 領導全球第一支數據科學團隊，至此數據科學越來越被廣泛流行，並應用到公共衛生、市場、金融、社會等各個領域。"""
raw_document_text = []

for sentence in document_text.split('\n'):
    if sentence:
        raw_document_text.append(sentence)
# raw_document_text


# 先處理資料
def get_summarization(sentences, top_n=5):
    sent2scores = {}
    non_appeared_words = set()
    # 計算分數
    for sentence in sentences:
        sent2scores[sentence] = 0
        words = list(jieba.cut(sentence))
        for word in words:
            if word in word2freq:
                sent2scores[sentence] += word2freq[word]
            else:
                non_appeared_words.add(word)
        # 標準化分數，不然越長的句子越占優勢
        sent2scores[sentence] /= len(words)
    
    top_n_sent2scores = sorted(sent2scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return top_n_sent2scores, non_appeared_words
    
    


top_n_sent2scores, non_appeared_words = get_summarization(raw_document_text, top_n=3)
for sent, score in top_n_sent2scores:
    print(f'句子重要性: /{score}/\n\n', sent)
    print('---'*30)

句子重要性: /0.13253603062084598/

 2001年，William S. Cleveland 提議將其設立為一個新的學科，吸收「計算在數據方面取得的進展」作為統計學的延伸。《Data Science Journal》及《The Journal of Data Science》分別於2002年與2003年發行；2005年9月，美國國家科學基金會發表了《長存的數位資料收集：使21世紀的研究與教育成為可能》，文中將數據科學家定義為「資訊與電腦科學家，資料庫與軟體工程師與程式設計師，跨學科專家，保管員以及專業注釋者，圖書館員，檔案館員和其他人員，這些人對數位資料收集的成功管理至關重要。」[8] 2008年 Jeff Hamerbatcher 與 DJ Patil circa 分別在 Facebook、LinkedIn 領導全球第一支數據科學團隊，至此數據科學越來越被廣泛流行，並應用到公共衛生、市場、金融、社會等各個領域。
------------------------------------------------------------------------------------------
句子重要性: /0.1165011459129106/

 1997年11月，吳建福 發表了題為「統計=資料科學？」的就任演講，[5] 因為他被任命為密西根大學的 H. C.卡弗教授。[6] 在這次演講中，他將統計工作描述為資料收集、資料建模和分析以及決策制定的三部曲。 在他的結論中，他開創了「資料科學」（而非「電腦科學」）這個術語的現代用法，並提倡將統計學重新命名為資料科學，而統計學家則應重新命名為資料科學家。[5] 後來，他發表了題為「統計=資料科學？」的演講，作為他1998年的首個 P. C. 馬哈拉諾比斯紀念演講。[7] 這些講座是為了紀念印度科學家、統計學家和印度統計學院​（英語）創始人P. C. 馬哈拉諾比斯​（英語）。
------------------------------------------------------------------------------------------
句子重要性: /0.09740764854629824/

 美國國家標準技術研究所於2015年發表七卷巨量資料參考框架（NIST Big Dat

> 可以看到詞頻基礎的缺陷會比較多，會被大量重複出現的詞帶走，但不一定是我們想要的摘要性意思，但不失為一個基本的方法！

### tf-idf

將步驟3 修改

3. 將中文句子、文字向量化。
    1. 透過corpus去計算 Count Vector，也就是每一個word，出現頻率。

4. 根據向量化分數取得摘要。
    1. 輸入為數個句子(一篇短文)，挑選出 top n的句子作為摘要。
    2. 其中每一個句子會有重要性分數，根據其每一個word的分數加總而成。(此處需要注意，這要特別小心處理贅詞，但這先忽略跳過)

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [69]:
for i in range(len(corpus)):
    # 打造tf-idf可用的格式
    corpus[i] = ' '.join(list(jieba.cut(corpus[i])))

corpus

['維基 百科 ， 自由 的 百科 全書',
 '（ 重新 導向 自資料 科學 ）',
 '跳至 導覽',
 '跳至 搜尋',
 '此條目 需要 擴充 。   ( 2013 年 7 月 15 日 ) 請 協助 改善 這篇 條目 ， 更進 一步 的 訊息 可能 會 在 討論 頁 或 擴充 請求 中 找到 。 請 在 擴 充條 目後將 此 模板 移除 。',
 '機器 學習 與 資料 探勘',
 '問題',
 '分類',
 '聚類',
 '回歸',
 '異常 檢測',
 '資料 淨化',
 '自動 機器 學習',
 '關聯 規則',
 '強化 學習',
 '結構 預測 （ 英語 ： Structured   prediction ）',
 '特徵 工程',
 '特徵學習',
 '線上 機器 學習',
 '無監督 學習',
 '半監督 學習 （ 英語 ： Semi - supervised   learning ）',
 '排序 學習 （ 英語 ： Learning   to   rank ）',
 '語法 歸納 （ 英語 ： Grammar   induction ）',
 '監督式 學習 ( 分類 \xa0 ·   回歸 )',
 '決策 樹',
 '整合 （ 裝袋 ， 提升 ， 隨機 森林 ）',
 'k - NN',
 '線性 回歸',
 '樸素 貝葉斯',
 '神經 網路',
 '邏輯 回歸',
 '感知器',
 '支持 向量 機 （ SVM ）',
 '相關 向量 機 （ RVM ）',
 '聚類',
 'BIRCH',
 '層次 （ 英語 ： Hierarchical   clustering ）',
 'k - 平均',
 '期望 最大化 （ EM ）',
 'DBSCAN',
 'OPTICS',
 '均值 飄移 （ 英語 ： Mean   shift ）',
 '降維',
 '因素 分析',
 'CCA',
 'ICA',
 'LDA',
 'NMF （ 英語 ： Non - negative   matrix   factorization ）',
 'PCA',
 'LASSO',
 't - SNE （ 英語 ： t - distributed   stochastic   neighbor   embedding ）',
 '結構 預測 （ 英

In [77]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

print(X.shape)
# del X

(5435, 10306)


In [99]:
sentences = []

for i in range(len(raw_document_text)):
    sent = ' '.join(list(jieba.cut(raw_document_text[i])))
    sentences.append(sent)
len(sentences)

5

In [100]:
sentences

['資料 科學 （ 英語 ： data   science ） 又 稱 數據 科學 ， 是 一門 利用 資料 （ 數據 ） 學習 知識 的 學科 ， 其目 標是 通過 從 資料 中 提取 出有 價值 的 部分 來 生產資料 產品 [ 1 ] ， 學科範圍 涵蓋 了 ： 資料 取得 、 資料 處理 、 資料 分析 等 過程 ， 舉凡 與 資料 有關 的 科學均 屬資料 科學 。',
 '資料 科學 結合 了 諸多 領域 中 的 理論 和 技術 ， 包括 應用 數學 、 統計 、 圖型 識別 、 機器 學習 、 資料視 覺化 、 資料 倉儲 以及 高效能 計算 。 資料 科學 通過 運用 各種 相關 的 資料 來 幫助 非 專業 人士 理解 問題 。   資料 科學 技術 可以 幫助 我們 如何 正確 的 處理 資料 並協助 我們 在 生物 學 、 社會 科學 、 人類 學等 領域 進行 研究 調研 。 此外 ， 資料 科學 也 對 商業 競爭 有 極大 的 幫助 [ 2 ] 。',
 '美國國家 標準 技術 研究所 於 2015 年 發表 七卷 巨量 資料 參考 框架 （ NIST   Big   Data   Reference   Architecture ， NBDRA ） ， 於 第一卷 定義篇 中將 資料 科學定 為 在 理論 科學 、 實驗 科學 和 計算 科學 之 後 的 第四 科學範式 [ 3 ] [ 4 ] 。 1960 年代 ， 彼得 · 諾爾 就 首次 提議 要 用 「 數據 科學 （ data   science 、 Datalogy ） 」 來 替代 「 計算 機科學 （ computer   science ） 」 ， 後 來 在 上 世紀 九十年代 中期 為國際 分類 社團 聯盟 所用 。',
 '1997 年 11 月 ， 吳建福   發表 了 題 為 「 統計 = 資料 科學 ？ 」 的 就任 演講 ， [ 5 ]   因為 他 被 任命 為 密西根 大學 的   H .   C . 卡弗 教授 。 [ 6 ]   在 這次 演講 中 ， 他將 統計 工作 描述 為 資料 收集 、 資料 建模 和 分析 以及 決策 制定 的 三部曲 。   在 他 的 結論 中 ， 他 開創 了 「 資料 科學 」 （ 而 非 「 電腦 科學 」 ）

In [103]:
def get_summarization_tfidf(sentences, vectorizer, top_n=5):
    sent2scores = {}
    # 計算分數
    tfidf_vec = vectorizer.transform(sentences)
    for i, sent in enumerate(sentences):
        sent2scores[sent] = 0
        sent2scores[sent] += np.sum(tfidf_vec[i])
    
    top_n_sent2scores = sorted(sent2scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return top_n_sent2scores

In [108]:
top_n_sent2scores_tfidf = get_summarization_tfidf(sentences, vectorizer, top_n=3)
for sent, score in top_n_sent2scores_tfidf:
    print(f'句子重要性: /{score}/\n\n', sent)
    print('---'*30)

句子重要性: /9.136883165543239/

 2001 年 ， William   S .   Cleveland   提議 將其 設立 為 一個 新 的 學科 ， 吸收 「 計算 在 數據 方面 取得 的 進展 」 作為 統計學 的 延伸 。 《 Data   Science   Journal 》 及 《 The   Journal   of   Data   Science 》 分別 於 2002 年 與 2003 年 發行 ； 2005 年 9 月 ， 美國國家 科學 基金 會 發表 了 《 長 存 的 數位 資料 收集 ： 使 21 世紀 的 研究 與 教育 成為 可能 》 ， 文中 將數據 科學家定 義為 「 資訊 與 電腦 科學家 ， 資料 庫 與 軟體 工程 師與 程式 設計師 ， 跨學科 專家 ， 保管 員 以及 專業 注釋 者 ， 圖書館員 ， 檔案 館員 和 其他人 員 ， 這些 人 對 數位 資料 收集 的 成功 管理 至關 重要 。 」 [ 8 ]   2008 年   Jeff   Hamerbatcher   與   DJ   Patil   circa   分別 在   Facebook 、 LinkedIn   領導 全球 第一支 數據 科學 團隊 ， 至此 數據 科學 越來 越 被 廣泛 流行 ， 並應 用到 公共 衛生 、 市場 、 金融 、 社會 等 各個 領域 。
------------------------------------------------------------------------------------------
句子重要性: /6.599525411107687/

 美國國家 標準 技術 研究所 於 2015 年 發表 七卷 巨量 資料 參考 框架 （ NIST   Big   Data   Reference   Architecture ， NBDRA ） ， 於 第一卷 定義篇 中將 資料 科學定 為 在 理論 科學 、 實驗 科學 和 計算 科學 之 後 的 第四 科學範式 [ 3 ] [ 4 ] 。 1960 年代 ， 彼得 · 諾爾 就 首次 提議 要 用 「 數據 科學 （ data   science 、 Datalogy ） 」 來 替代 「 計算 機科學 （ 